In [8]:
from openpyxl import Workbook, load_workbook
from fuzzywuzzy import process, fuzz
import time
import pandas as pd
import numpy as np

In [9]:
from szp_funcs import print_df

In [10]:
path_to_dir = 'C:/Users/PetukhovMD/Desktop/Направления и категории/'
file = 'новые данные в выгрузках.xlsx'
pages = ['job_cat', 'job_direc', 'source']

In [11]:
categories = pd.read_excel(path_to_dir + file, sheet_name= pages[0])
direction = pd.read_excel(path_to_dir + file, sheet_name= pages[1])
source = pd.read_excel(path_to_dir + file, sheet_name= pages[2])

In [12]:
jobs_source = source['job'].unique().tolist()
jobs_no_direction = source[source['direc'].isna()]['job'].unique().tolist()
directions_source = source['direc'].unique().tolist()

In [13]:
def matcher(row, look_up, name):
    ans = process.extractOne(row[name], look_up, scorer=fuzz.WRatio, score_cutoff=60)
    if ans:
        return ans[0]
    return ans

In [14]:
categories['real_job'] = categories.apply(lambda row: matcher(row, jobs_source, 'job'), axis=1)
direction['real_job'] = direction.apply(lambda row: matcher(row, jobs_source, 'job'), axis=1)

In [15]:
out_cat_by_job = categories.drop('job', axis=1).groupby('real_job').agg({
    'job_cat': 'unique'
}).reset_index()

In [16]:
def to_good_str(row, name):
    s = ''
    for i in  row[name]:
        s += str(i) + '&'
    return s[:-1]

In [17]:
out_cat_by_job['job_cat_str'] = out_cat_by_job.apply(lambda row: to_good_str(row, 'job_cat'), axis=1)

In [18]:
print_df(out_cat_by_job.drop('job_cat', axis=1), path_to_dir + 'категории по должностям.xlsx')

In [19]:
direction[(direction['real_job'].isin(jobs_no_direction)) & (direction['job_direc'].isna() == False)]

,job,job_direc,real_job
134,Преподаватель,производственной,Преподаватель
139,Специалист,учебной части,Специалист
142,Специалист,в сфере ИКТ,Специалист
212,Ведущий специалист,по финансово-экономической деятельности,Ведущий специалист
222,Специалист,по материально-техническому обеспечению,Специалист
223,Специалист,по здоровьесбережению и профилактике травматизма,Специалист
224,Специалист,по питанию,Специалист
243,Ведущий специалист,в сфере закупок,Ведущий специалист
244,Мастер производственного обучения,учебно-производственной мастерской,Мастер производственного обучения
245,Ведущий специалист,по административно-хозяйственному обеспечению,Ведущий специалист


In [20]:
print_df(direction[direction['job_direc'].isin(directions_source + [np.nan]) == False][['real_job', 'job_direc']], path_to_dir + 'некорректные направления по должностям')

In [21]:
categories

,job,job_cat,real_job
0,Преподаватель,Педагогические работники дошкольных образовате...,Преподаватель
1,Преподаватель,Преподаватели образовательных учреждений СПО,Преподаватель
2,Библиотекарь,Прочий персонал,Библиотекарь
3,Специалист,Прочий персонал,Специалист
4,Педагог-организатор,Педагогические работники образовательных учреж...,Педагог-организатор
...,...,...,...
1029,Секретарь по учебной части,NaN,Секретарь учебной части
1030,Начальник отдела,Административно-управленческий,Начальник отдела
1031,Руководитель,Руководитель организации,Руководитель
1032,Консультант,NaN,Консультант


In [22]:
direction

,job,job_direc,real_job
0,Преподаватель,NaN,Преподаватель
1,Библиотекарь,NaN,Библиотекарь
2,Специалист,NaN,Специалист
3,Педагог-организатор,NaN,Педагог-организатор
4,Методист,NaN,Методист
...,...,...,...
734,Переводчик-дактилолог (сурдопереводчик),NaN,Переводчик-дактилолог
735,Секретарь по учебной части,NaN,Секретарь учебной части
736,Заместитель начальника отдела,по административно-хозяйственной работе,Заместитель начальника отдела
737,Тьютор,начальных классов,Тьютор
